In [5]:

import pandas as pd

# Your table data as lists
bins = ["0-5", "6-11", "12-20", "21-24", "25-40", "41-59", "60-120"]
total_renewals = [10000, 20000, 30000, 40000, 50000, 60000, 70000]
promo_renewals = [0.0, 0.2, 0.2, 0.0, 0.0, 0.0,0.0]
nonpromo_renewals = [0.0, 0.1, 0.1, 0.0, 0.0, 0.0,0.0]
 
def unroll_rates(bins, total_renewals, promo_renewals, nonpromo_renewals):
    # Parse bins and create a DataFrame
    terms = []
    assigned_rates = []
    promo_rates = []
    nonpromo_rates = []
    
    for i,(bin_range, rate, promo, non_promo) in enumerate(zip(bins, total_renewals, promo_renewals, nonpromo_renewals)):
        start, end = map(int, bin_range.split('-'))
  
         # For all bins except the last, include the entire range
        terms.extend(range(start, end + 1))
        assigned_rates.extend([rate] * (end - start + 1))
        promo_rates.extend([promo] * (end - start + 1))
        nonpromo_rates.extend([non_promo] * (end - start + 1))
 
            
    # Create a DataFrame
    df = pd.DataFrame({
        'term': terms,
        'total_renewal': assigned_rates,
        'promo': promo_rates,
        'non_promo': nonpromo_rates
    })

    return df

df = unroll_rates(bins,  total_renewals, promo_renewals, nonpromo_renewals)


In [12]:
import pandas as pd

def unroll_rates(bins, total_renewals, promo_renewals, nonpromo_renewals, special_numbers, allocation):
    # Initialize lists
    terms, assigned_rates, promo_rates, nonpromo_rates, special_alloc, bin_field = [], [], [], [], [], []

    for bin_range, rate, promo, non_promo in zip(bins, total_renewals, promo_renewals, nonpromo_renewals):
        start, end = map(int, bin_range.split('-'))
        bin_terms = range(start, end + 1)

        # Calculate number of special numbers in this bin
        special_in_bin = [num for num in special_numbers if start <= num <= end]
        count_special = len(special_in_bin)
        count_others = (end - start + 1) - count_special
        other_allocation = (1 - allocation) / count_others if count_others > 0 else 0

        # Assign values to lists
        for term in bin_terms:
            terms.append(term)
            assigned_rates.append(rate)
            promo_rates.append(promo)
            nonpromo_rates.append(non_promo)
            special_alloc.append(allocation if term in special_in_bin else other_allocation)
            bin_field.append(bin_range)
            print(bin_range)

    # Create a DataFrame
    df = pd.DataFrame({
        'term': terms,
        'total_renewal': assigned_rates,
        'promo': promo_rates,
        'non_promo': nonpromo_rates,
        'special_allocation': special_alloc,
        'bin': bin_field
    })

    return df

# Example usage
bins = ["0-5", "6-11"] #  "12-20", "21-24", "25-40", "41-59", "60-120"]
total_renewals = [10000, 20000, 30000, 40000, 50000, 60000, 70000]
promo_renewals = [0.0, 0.2, 0.2, 0.0, 0.0, 0.0, 0.0]
nonpromo_renewals = [0.0, 0.1, 0.1, 0.0, 0.0, 0.0, 0.0]
special_numbers = [6, 7, 12, 13]
allocation = 0.9

df = unroll_rates(bins, total_renewals, promo_renewals, nonpromo_renewals, special_numbers, allocation)


0-5
0-5
0-5
0-5
0-5
0-5
6-11
6-11
6-11
6-11
6-11
6-11


In [14]:
df.groupby('bin')['special_allocation'].sum()

bin
0-5     0.1
6-11    1.9
Name: special_allocation, dtype: float64

In [49]:
import pandas as pd
import numpy as np

def unroll_rates(bins, total_renewals, promo_renewals, nonpromo_renewals, special_numbers, allocation):
    terms, assigned_rates, promo_rates, nonpromo_rates, special_alloc, bin_field = [], [], [], [], [], []

    for bin_range, rate, promo, non_promo in zip(bins, total_renewals, promo_renewals, nonpromo_renewals):
        start, end = map(int, bin_range.split('-'))
        bin_terms = range(start, end + 1)

        # Find special numbers in this bin
        special_in_bin = [num for num in special_numbers if start <= num <= end]
        count_special = len(special_in_bin)
        count_others = len(bin_terms) - count_special

        # Allocate percentages
        if count_special > 0:
            special_number_allocation = allocation / count_special
            other_allocation = (1 - allocation) / count_others if count_others > 0 else 0
        else:
            special_number_allocation = 0
            other_allocation = 1 / len(bin_terms)

        # Assign values to lists
        for term in bin_terms:
            terms.append(term)
            assigned_rates.append(rate)
            promo_rates.append(promo)
            nonpromo_rates.append(non_promo)
            alloc = special_number_allocation if term in special_in_bin else other_allocation
            special_alloc.append(alloc)
            bin_field.append(bin_range)

    # Create a DataFrame
    df = pd.DataFrame({
        'term': terms,
        'total_renewal': assigned_rates,
        'promo': promo_rates,
        'non_promo': nonpromo_rates,
        'special_allocation': special_alloc,
        'bin': bin_field
    })

    return df

# Example usage
bins = ["1-5", "6-11", "12-20", "21-24", "25-40", "41-59", "60-120"]
total_renewals = [10000, 20000, 30000, 40000, 50000, 60000, 70000]
promo_renewals = [0.0, 0.2, 0.2, 0.0, 0.0, 0.0, 0.0]
nonpromo_renewals = [0.0, 0.1, 0.1, 0.0, 0.0, 0.0, 0.0]
promos = np.array( [[0, 6, 7, 12, 13], 
                    [5, 6, 8, 12, 13]]) #  ([5, 6, 7, 12, 13], [5, 6, 7, 12, 13]))
allocation = 0.9
perc = np.empty((special_numbers.shape[0],120))
for period in range(promos.shape[0]):
    df = unroll_rates(bins, total_renewals, promo_renewals, nonpromo_renewals, promos[period], allocation)
    perc[period] = df['special_allocation'].values
# Test the results
print(df.groupby('bin')['special_allocation'].sum())
pd.DataFrame(perc)#.shape
df.head()

bin
1-5       1.0
12-20     1.0
21-24     1.0
25-40     1.0
41-59     1.0
6-11      1.0
60-120    1.0
Name: special_allocation, dtype: float64


,term,total_renewal,promo,non_promo,special_allocation,bin
0,1,10000,0.0,0.0,0.025,1-5
1,2,10000,0.0,0.0,0.025,1-5
2,3,10000,0.0,0.0,0.025,1-5
3,4,10000,0.0,0.0,0.025,1-5
4,5,10000,0.0,0.0,0.900,1-5


In [78]:
import pandas as pd
import numpy as np

def unroll_percentage(bins,  special_numbers, allocation):
    terms,  special_alloc, bin_field = [], [], [] 
    
    for bin_range  in bins:
       
        start, end =  map(int, bin_range.split('-'))
        start = max(1,start)
        bin_terms = range(start, end + 1)

        # Find special numbers in this bin
        special_in_bin = [num for num in special_numbers if start <= num <= end]
        count_special = len(special_in_bin)
        count_others = len(bin_terms) - count_special

        # Allocate percentages
        if count_special > 0:
            special_number_allocation = allocation / count_special
            other_allocation = (1 - allocation) / count_others if count_others > 0 else 0
        else:
            special_number_allocation = 0
            other_allocation = 1 / len(bin_terms)

        # Assign values to lists
        for term in bin_terms:
            terms.append(term) 
            alloc = special_number_allocation if term in special_in_bin else other_allocation
            special_alloc.append(alloc)
            bin_field.append(bin_range)

    # Create a DataFrame
    df = pd.DataFrame({
        'term': terms, 
        'special_allocation': special_alloc,
        'bin': bin_field
    })

    return df

def nbv_promo_allocation(bins, promos, allocation):
    size = int(bins[-1].split('-')[1])
    perc = np.empty((special_numbers.shape[0],size))
    for period in range(promos.shape[0]):
        df = unroll_percentage( bins, promos[period], allocation)
        perc[period] = df['special_allocation'].values
    return perc
 
bins = ["0-5", "6-11", "12-20", "21-24", "25-40", "41-59", "60-120"] 
promos = np.array( [[-1, 6, 7, 12, 13], 
                    [5, 6, 8, 12, 13]]) #  ([5, 6, 7, 12, 13], [5, 6, 7, 12, 13]))
allocation = 0.9

perc = nbv_promo_allocation(bins, promos, allocation)

# Test the results
# print(df.groupby('bin')['special_allocation'].sum())
pd.DataFrame(perc)


,0,1,2,3,4,5,6,7,8,9,...,110,111,112,113,114,115,116,117,118,119
0,0.200,0.200,0.200,0.200,0.2,0.45,0.450,0.025,0.025,0.025,...,0.016393,0.016393,0.016393,0.016393,0.016393,0.016393,0.016393,0.016393,0.016393,0.016393
1,0.025,0.025,0.025,0.025,0.9,0.45,0.025,0.450,0.025,0.025,...,0.016393,0.016393,0.016393,0.016393,0.016393,0.016393,0.016393,0.016393,0.016393,0.016393
